# $\fbox{2-4}$ 傷病条件付き診療行為

### 準備 & 作成したデータの読み込み

In [1]:
# グラフを移動拡大縮小したい場合
%matplotlib notebook
# グラフをインラインで表示
# %matplotlib inline

import gc

import mylibs.mylib1 as mylib1  # 本書 1 章の関数群を含む自作ライブラリ
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm

# 表示する行・列の数を設定
pd.options.display.max_rows = 6
pd.options.display.max_columns = 10

# matplotlib のフォントを設定
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'

In [2]:
df_info = pd.read_csv('./pseudo_medical/records/excl_bp/info.csv')
df_commons = pd.read_csv('./pseudo_medical/records/excl_bp/commons.csv')
df_diseases = pd.read_csv('./pseudo_medical/records/excl_bp/diseases.csv')
df_treatments = pd.read_csv('./pseudo_medical/records/excl_bp/treatments.csv')

# 分析開始時点 (2010年1月)
start_study_t = 2010
# 分析終了時点 (2019年12月)
end_study_t = 2019.999

In [3]:
print(len(df_info))
df_info.head()

4815


,iid,sex,family,birth_ym,start_obs_ym,end_obs_ym,death
0,i000000,M,1,1952/06,2010/01,2013/09,0
1,i000001,F,2,1972/11,2010/01,2019/12,0
2,i000002,M,1,1978/03,2010/01,2016/09,0
3,i000003,M,2,1989/03,2010/01,2013/12,0
4,i000006,M,1,1973/11,2010/04,2013/09,0


In [4]:
print(len(df_commons))
df_commons.head()

433313


,iid,rid,ym,receipt_type,admission_ym,days
0,i000000,r00000001,2010/01,outpatient,-,1
1,i000000,r00000002,2010/02,outpatient,-,1
2,i000000,r00000003,2010/02,outpatient,-,1
3,i000000,r00000004,2010/03,outpatient,-,1
4,i000000,r00000005,2010/04,outpatient,-,1


In [5]:
print(len(df_diseases))
df_diseases.head()

433313


,iid,rid,first_ym,icd10_code
0,i000000,r00000001,2010/01,D50
1,i000000,r00000002,2010/02,H60
2,i000000,r00000003,2010/02,S00
3,i000000,r00000004,2010/03,G00
4,i000000,r00000005,2010/04,C00


In [6]:
print(len(df_treatments))
df_treatments.head()

868945


,iid,rid,treatment_code
0,i000000,r00000001,A000
1,i000000,r00000001,F000
2,i000000,r00000002,A000
3,i000000,r00000002,F000
4,i000000,r00000003,A000


### データの絞り込み

In [7]:
# ここでは、がんの定義を ICD-10 の C00-D09 とする。
in_cancer = ('C00' <= df_diseases['icd10_code']) & \
            (df_diseases['icd10_code'] <= 'D09')
df_cancers = df_diseases[in_cancer]
# 手術 K コードを含むデータのみに絞り込む
contain_K = df_treatments['treatment_code'].str.contains('K')
df_surgs = df_treatments[contain_K]
# rid をキーに inner join する
df_cancer_surgs = pd.merge(df_cancers, df_surgs, on=[
                           'iid', 'rid'], how='inner')

print(len(df_cancers), len(df_surgs), len(df_cancer_surgs))
df_cancer_surgs.head()

21677 4375 231


,iid,rid,first_ym,icd10_code,treatment_code
0,i000027,r00001264,2010/05,C00,K000
1,i000041,r00001991,2012/04,C00,K000
2,i000056,r00002877,2016/09,C00,K000
3,i000068,r00003915,2015/04,C00,K000
4,i000126,r00006614,2011/07,C00,K000


### がん入院・がん外来

In [8]:
# がん入院
df_commons_inpatient = df_commons[df_commons['receipt_type'] == 'inpatient']
df_cancer_inpatient = pd.merge(df_cancers, df_commons_inpatient, on=[
                               'iid', 'rid'], how='inner')

print(len(df_cancer_inpatient))
df_cancer_inpatient.head()

408


,iid,rid,first_ym,icd10_code,ym,receipt_type,admission_ym,days
0,i000041,r00001967,2010/12,C00,2010/12,inpatient,2010/12,15
1,i000041,r00001968,2010/12,C00,2011/01,inpatient,2010/12,8
2,i000041,r00001991,2012/04,C00,2012/04,inpatient,2012/04,15
3,i000041,r00001992,2012/04,C00,2012/05,inpatient,2012/04,11
4,i000162,r00008666,2019/01,C00,2019/01,inpatient,2019/01,10


In [9]:
# がん外来
df_commons_outpatient = df_commons[df_commons['receipt_type'] == 'outpatient']
df_cancer_outpatient = pd.merge(df_cancers, df_commons_outpatient, on=[
                                'iid', 'rid'], how='inner')

print(len(df_cancer_outpatient))
df_cancer_outpatient.head()

21269


,iid,rid,first_ym,icd10_code,ym,receipt_type,admission_ym,days
0,i000000,r00000005,2010/04,C00,2010/04,outpatient,-,1
1,i000000,r00000015,2011/02,C00,2011/02,outpatient,-,1
2,i000000,r00000022,2011/04,C00,2011/04,outpatient,-,1
3,i000000,r00000027,2011/08,C00,2011/08,outpatient,-,1
4,i000000,r00000031,2011/11,C00,2011/11,outpatient,-,1


## まとめ